# 12. Inheritance: for good or for worse 

## 12.1 Subclassing built-in types is tricky

- Python 2.2 이전에는 list나 dict등의 built-in type을 subclass하는 것 자체가 불가능했음
- 공식적으로 CPython은 built-in type의 subclass에서 override한 method가 언제 호출되는지, 호출되지 않는지에 대해 명확한 규칙을 정의하지 않음
- 일반적으로 subclass에서 override한 method는 같은 object의 다른 built-in method에 의해 호출되지 않음
    - ex: dict의 subclass에서 override한 __getitem__() method는 get()과 같은 다른 built-in method에 의해 호출되지 않음

In [2]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

dd = DoppelDict(one=1) # dict의 __init__() method는 __setitem__()이 override되었다는 사실을 무시. one:[1,1]이 아닌 one: 1을 저장
print(dd)
dd['two']=2 # []연산자는 override한 __setitem__()을 호출. two:[2,2]를 저장
print(dd)
dd.update(three=3) # dict의 update method도 override된 __setitem__()d을 호출 안함. --> three:3을 저장
print(dd)

{'one': 1}
{'one': 1, 'two': [2, 2]}
{'one': 1, 'two': [2, 2], 'three': 3}


In [4]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42
dd = AnswerDict(a='foo')
print(dd['a']) # []연산자는 override한 __getitem__()을 호출. 42를 반환
d = {}
d.update(dd) # d는 평범한 dict 객체. update를 통해, d를 update
print(d['a']) # 그러나, dict.update() method는 override된 AnswerDict.__getitem__() method를 무시
print(d)

42
foo
{'a': 'foo'}


- 이와 같이, dict, list, str등의 built-in type을 사용하면, 사용자가 정의한 override method를 무시.
- 이 클래스들 직접 상속받지 말자.
- 대신, 쉽게 확장이 가능한 collections module의 UserDict등을 사용할 것!

In [5]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

dd = DoppelDict2(one=1)
print(dd)
dd['two'] = 2
print(dd)
dd.update(three=3)
print(dd)

        

{'one': [1, 1]}
{'one': [1, 1], 'two': [2, 2]}
{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}


In [7]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42
    
ad = AnswerDict2(a='foo')
print(ad['a'])
d = dict()
d.update(ad)
print(d['a'])
print(d)

42
42
{'a': 42}


## 12.2 Multiple Inheritance and method determining order
- Multiple Inheritance를 지원하는 언어에서는 별개의 상위 클래스가 동일 이름으로 method를 구현할 때 발생하는 이름 충돌 문제를 해결해야 함
    - Example: Diamond Problem


In [25]:
class A:
    def ping(self):
        print('ping: ', self)

class B(A):
    def pong(self):
        print('pong_b: ', self)

class C(A):
    def pong(self):
        print('pong_c: ', self)

class D(B,C):
    def ping(self):
        super().ping()
        print('post-ping: ', self)
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)
        
class E1(C,B):
    def ping(self):
        super().ping() # Super class에 위임할 땐, built-in function인 super()를 사용하는 거이 좋다.
        print('post-ping: ', self)
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)
        
class E2(C,B):
    def ping(self):
        A.ping(self) # MRO를 우회하여, super class method를 직접 호출하고자 할 떄 사용!
        print('post-ping: ', self)
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [26]:
d = D()
d.pong()
D.pong(d)
C.pong(d)

pong_b:  <__main__.D object at 0x7f63583f7c50>
pong_b:  <__main__.D object at 0x7f63583f7c50>
pong_c:  <__main__.D object at 0x7f63583f7c50>


In [18]:
# MRO: Method Resolution Order(메소드 결정 순서)
print(D.__mro__) 
print(E1.__mro__)
print(E2.__mro__)

(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)
(<class '__main__.E1'>, <class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)
(<class '__main__.E2'>, <class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)


In [27]:
# MRO우회시
e1 = E1()
e2 = E2()
e1.ping()
e2.ping()

ping:  <__main__.E1 object at 0x7f63583f7b00>
post-ping:  <__main__.E1 object at 0x7f63583f7b00>
ping:  <__main__.E2 object at 0x7f63583f7780>
post-ping:  <__main__.E2 object at 0x7f63583f7780>


In [32]:
# super()는 MRO를 따른다
d = D()
d.ping()

ping:  <__main__.D object at 0x7f63583f7b38>
post-ping:  <__main__.D object at 0x7f63583f7b38>


In [33]:
d.pingpong()

ping:  <__main__.D object at 0x7f63583f7b38>
post-ping:  <__main__.D object at 0x7f63583f7b38>
ping:  <__main__.D object at 0x7f63583f7b38>
pong_b:  <__main__.D object at 0x7f63583f7b38>
pong_b:  <__main__.D object at 0x7f63583f7b38>
pong_c:  <__main__.D object at 0x7f63583f7b38>


- d.pingpong()의 결과
    - ping:  <__main__.D object at 0x7f63583f7b38> --> self.ping()에서 나온 결과
    - post-ping:  <__main__.D object at 0x7f63583f7b38> --> self.ping()에서 나온 결과
    - ping:  <__main__.D object at 0x7f63583f7b38> --> super().ping()에서 나온 결과로 A.ping()에서 나온 결과
    - pong_b:  <__main__.D object at 0x7f63583f7b38> --> self.pong()에서 나온 결과로 B.pong()에서 나온 결과
    - pong_b:  <__main__.D object at 0x7f63583f7b38> --> super().pong()에서 나온 결과로 B.pong()에서 나온 결과
    - pong_c:  <__main__.D object at 0x7f63583f7b38> --> C.pong(self)에서 나온 결과

In [37]:
# __mro__from various classes
import io, numbers, tkinter

print(bool.__mro__)
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))
print_mro(bool)
print_mro(numbers.Integral)
print_mro(io.BytesIO)
print_mro(io.TextIOWrapper)

(<class 'bool'>, <class 'int'>, <class 'object'>)
bool, int, object
Integral, Rational, Real, Complex, Number, object
BytesIO, _BufferedIOBase, _IOBase, object
TextIOWrapper, _TextIOBase, _IOBase, object


In [38]:
print_mro(tkinter.Text)
print_mro(tkinter.Toplevel)
print_mro(tkinter.Widget)
print_mro(tkinter.Button)
print_mro(tkinter.Entry)
print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object
Toplevel, BaseWidget, Misc, Wm, object
Widget, BaseWidget, Misc, Pack, Place, Grid, object
Button, Widget, BaseWidget, Misc, Pack, Place, Grid, object
Entry, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, object
Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object
